OpenVINO Local Pipelines with HuggingFace

https://python.langchain.com/docs/integrations/llms/openvino

In [ ]:
%pip install --upgrade-strategy eager "optimum[openvino,nncf]" --quiet



In [1]:
## login to huggingfacehub to get access to pretrained model 

from huggingface_hub import notebook_login, whoami

try:
    whoami()
    print('Authorization token already provided')
except OSError:
    notebook_login()

Authorization token already provided


https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

In [2]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

ov_llm = HuggingFacePipeline.from_model_id(
    #model_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_id="gpt2",
    task="text-generation",
    backend="openvino",
    model_kwargs={"device": "GPU", "ov_config": ov_config},
    pipeline_kwargs={"max_new_tokens": 512},
)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


Framework not specified. Using pt to export the model.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.2.2+cpu
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.
Overriding 1 configuration item(s)
	- use_cache -> True
c:\workspace\llm\.venv\lib\site-packages\transformers\modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
c:\workspace\llm\.venv\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:801: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value w

In [5]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

"""
prompt = PromptTemplate.from_template(template)

chain = prompt | ov_llm

question = "i can fly and i can swim, what am i?"

from pprint import pprint

pprint(chain.invoke({"question": question}))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


('Question: what do you like?\n'
 '\n'
 '\n'
 'The main character is pretty good, but his appearance is very off center so '
 'I would advise him not to speak very much, but if you want to be honest the '
 'main character speaks with a little more calm demeanor...')
